#### Imports

In [1]:
import PyPDF2
import csv
import re
import json
import os
import us

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [2]:
year = 2003
pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 742


In [4]:
def clean_key_page(page, offset):
    raw_abbr_key = pdfReader.getPage(page - 1).extractText()
    m = re.findall(r'[A-Z]{3}', raw_abbr_key)[offset:]
    return [l[:2] for l in m]

In [5]:
abbr_key = ['MH-SA']
abbr_key += clean_key_page(5, 4)

In [6]:
for i in range(1, 68):
    abbr_key.append('N' + str(i))

In [7]:
for i in range(1, 114):
    abbr_key.append('F' + str(i))

In [8]:
abbr_key

['MH-SA',
 'SA',
 'MH',
 'SA',
 'GH',
 'TX',
 'DT',
 'MM',
 'LA',
 'DM',
 'LA',
 'HH',
 'OP',
 'PH',
 'RR',
 'HI',
 'AD',
 'CO',
 'HV',
 'HI',
 'AI',
 'GL',
 'SE',
 'PW',
 'WO',
 'MN',
 'DU',
 'DW',
 'CJ',
 'MD',
 'MC',
 'PI',
 'MI',
 'TR',
 'CA',
 'SF',
 'SS',
 'PA',
 'AH',
 'SP',
 'N1',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'N10',
 'N11',
 'N12',
 'N13',
 'N14',
 'N15',
 'N16',
 'N17',
 'N18',
 'N19',
 'N20',
 'N21',
 'N22',
 'N23',
 'N24',
 'N25',
 'N26',
 'N27',
 'N28',
 'N29',
 'N30',
 'N31',
 'N32',
 'N33',
 'N34',
 'N35',
 'N36',
 'N37',
 'N38',
 'N39',
 'N40',
 'N41',
 'N42',
 'N43',
 'N44',
 'N45',
 'N46',
 'N47',
 'N48',
 'N49',
 'N50',
 'N51',
 'N52',
 'N53',
 'N54',
 'N55',
 'N56',
 'N57',
 'N58',
 'N59',
 'N60',
 'N61',
 'N62',
 'N63',
 'N64',
 'N65',
 'N66',
 'N67',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'F17',
 'F18',
 'F19',
 'F20',
 'F21',
 'F22',
 'F23',
 'F24',
 'F25',

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [9]:
def split_data(rawdata):
    '''
    Roughly split data, returns list of list.
    '''
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        #if i is the last line of keys, i+1 is facility name, i-1 is the phone number or other line of keys
        if (rawdata[i].split()[0] in abbr_key) and (not rawdata[i+1].split()[0] in abbr_key): 
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data

In [36]:
def clean_data(cns, year, pagen):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    #deal with first center
    data_holder = [year, pagen]
    cn = cns[0]
    cn_sn_all = ''
    i = 0
    while i < len(cn):
        if ',' in cn[i]:
            break
        cn_sn_all += ' ' + cn[i]
        i += 1
    cn_sn_all = cn_sn_all.strip()

    m = re.finditer(r'[A-Z][A-Z]+', cn_sn_all)
    cn_sn = ''
    for it in m:
        temp_upper = cn_sn_all[it.start():it.end()-1]
        a = us.states.lookup(temp_upper)
        if a != None:
            cn_sn = cn_sn_all[it.end() - 1:]
            break
    data_holder += split_center_street(cn_sn)
    
    #City, State, Postal_code
    city, state_postcode = cn[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    
    #phone + keys
    phone = cn[i]
    i += 1
    while not cn[i][0:2].isupper():
        phone += ' ' + cn[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(cn[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(cn[i:]))
            
    #clean the keys in last line also split the other center name if present
    keys_tail = data_holder[-1]
    i = 0
    keys = keys_tail.split(' ')
    print(keys)
    print(data_holder)
    
    #i = 1
    #while i < len(dcs):
        
    #    i += 1

In [31]:
rawdata[0]

['16',
 'For Code Definitions - See',
 'KEY on Page v. and cover flap',
 'ARIZONATerros Inc',
 '3118 East McDowell Road',
 'Phoenix, AZ 85008',
 '(602) 685-6000',
 'MH-SA TX DT OP CO HV WO MN CJ SFMD PI MI SS SP F46Terros Inc']

In [25]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list
    '''
    #Exception to split at number :
        #When it ends with "P.O. Box",
        #When the number being found is at the very front (wrongly included page number or 
        #center name starts with number)
        
    #If the number is at the end(suite xxx or building xxx) lets include the word in front of it)
    #print(cn_sa)
    if cn_sa[0].isnumeric():
        cn_sa[0].replace(' For Code Definitions - See KEY on Page v. and cover flap', '')
    
            
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if cn_sa[span[0] - 9:span[0] - 1] == 'P.O. Box':
                return ([cn_sa[:span[0] - 10], cn_sa[span[0] - 9:]])
            if span[1] != len(cn_sa):
                return ([cn_sa[:span[0] - 1], cn_sa[span[0]:]])
            if span[1] == len(cn_sa):
                return ([' '.join(cn_sa.split()[:-2]), ' '.join(cn_sa.split()[-2:])])
    return [cn_sa, '']

In [37]:
pagen = 30
page = pdfReader.getPage(pagen - 1).extractText()
rawdata = page.split('\n')
rawdata[:] = [l.replace('  ', ' ').strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]

#split
rawdata = split_data(rawdata)
#rawdata
#clean
clean_data(rawdata, 2003, pagen)

['MH-SA', 'TX', 'DT', 'OP', 'CO', 'HV', 'WO', 'MN', 'CJ', 'SFMD', 'PI', 'MI', 'SS', 'SP', 'F46Terros', 'Inc']
[2003, 30, 'Terros Inc', '3118 East McDowell Road', 'Phoenix', 'AZ', '85008', '(602) 685-6000', 'MH-SA TX DT OP CO HV WO MN CJ SFMD PI MI SS SP F46Terros Inc']


In [ ]:
def write_data(year, page, dc, writer):
    '''
    Writes data in dc, returns appending data if any
    '''
    i = 0
    data_holder = [year, page]
    #Center name, street address
    cn_sa = ''
    while (',' not in dc[i]) or not dc[i][-5:].isnumeric(): #(len(dc[i]) < 2) or 
        cn_sa += ' ' + dc[i]
        i += 1
    data_holder += split_center_street(cn_sa.strip(), page)

    #City, State, Postal_code
    city, state_postcode = dc[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    #phone + keys
    phone = dc[i]
    i += 1
    while not dc[i][0:2].isupper():
        phone += ' ' + dc[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(dc[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(dc[i:]))
        
    if not data_holder[-1].isupper(): #if the last line of keys also contains another center
        keys_tail = data_holder[-1]
        tail_start = re.search(r'[A-Z][a-z]', keys_tail).start()
        data_holder[-1] = keys_tail[: tail_start]
        return keys_tail[tail_start:]
        
    #write data
    #print(data_holder)
    writer.writerow(data_holder)

In [ ]:
def run(year, start_page, end_page = 0):
    
    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1
    
    #open/create csv
    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])

    for i in range(start_page, end_page + 1):
        page = pdfReader.getPage(i).extractText()
        rawdata = page.split('\n')
        
        
        rawdata[:] = [l.replace('˜', ' ').replace('  ', ' ').replace('  ', ' ').strip() for l in rawdata \
                      if not (l.strip() == '' or l == 'KEY')]
        #print(rawdata)
        if (len(rawdata) > 3):
            data = split_data(rawdata)
            clean_facility_name(data)
            #return data
            with open(save_as_filename, 'a', newline='') as file:
                writer = csv.writer(file)
                for dc in data:
                    write_data(year, i + 1, dc, writer)

In [ ]:
run(2007, 613) #355, 364, 365, 394, 613